# Screening Round (QOSF Mentorship) [Cohort 7]

#### Importing libraries

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute

### Function to check whether the 4 integers represent sides of a rectangle

In [4]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute

def check_rectangle_exists(A, B, C, D):
    # Initialize the quantum and classical registers
    qr = QuantumRegister(6, 'q')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(qr, cr)

    # Encode the input values onto the qubits
    for i, val in enumerate([A, B, C, D]):
        for j in range(val):
            qc.x(qr[i])

    # Check if there exists a rectangle
    qc.cx(qr[0], qr[3])
    qc.cx(qr[1], qr[4])
    qc.cx(qr[2], qr[5])
    qc.ccx(qr[0], qr[1], qr[2])
    qc.ccx(qr[3], qr[4], qr[5])
    qc.ccx(qr[0], qr[4], qr[5])
    qc.ccx(qr[3], qr[1], qr[2])
    qc.measure(qr[5], cr[0])

    # Execute the circuit and get the results
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1)
    result = job.result().get_counts(qc)

    # Return 1 if there exists a rectangle, otherwise 0
    return 1 if '1' in result else 0


print(check_rectangle_exists(5,6,6,5))

0


In [5]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit.providers.aer import QasmSimulator

def rectangle_exists(A, B, C, D):
    # Determine the maximum value of the input integers
    max_val = max(A, B, C, D)
    
    # Determine the number of qubits required to represent the integers
    num_qubits = max_val.bit_length()
    
    # Initialize quantum and classical registers
    qreg = QuantumRegister(num_qubits, 'q')
    creg = ClassicalRegister(4, 'c')
    
    # Initialize quantum circuit
    qc = QuantumCircuit(qreg, creg)
    
    # Encode the integers in binary on the quantum register
    for i, val in enumerate([A, B, C, D]):
        for j in range(num_qubits):
            if (val >> j) & 1:
                qc.x(qreg[j + i * num_qubits])
    
    # Check if there is a rectangle
    qc.cx(qreg[0], qreg[num_qubits])
    qc.cx(qreg[2 * num_qubits], qreg[num_qubits])
    qc.ccx(qreg[num_qubits], qreg[2 * num_qubits], qreg[3 * num_qubits])
    qc.measure(qreg[num_qubits: 4 * num_qubits], creg)
    
    # Simulate the circuit and measure the results
    backend = QasmSimulator()
    job = execute(qc, backend=backend, shots=1024)
    results = job.result().get_counts(qc)
    
    # Check if the output indicates that a rectangle exists
    return int('0001' in results)

print(rectangle_exists(5,6,6,5))

IndexError: list index out of range

In [6]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer

def is_rectangle_possible(A, B, C, D):
    # Determine the number of qubits needed
    max_side_length = max(A, B, C, D)
    num_qubits = max_side_length.bit_length() + 1
    
    # Create the quantum and classical registers
    qr = QuantumRegister(num_qubits, 'q')
    cr = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Encode the side lengths as binary numbers
    A_bin = format(A, f'0{num_qubits}b')[::-1]
    B_bin = format(B, f'0{num_qubits}b')[::-1]
    C_bin = format(C, f'0{num_qubits}b')[::-1]
    D_bin = format(D, f'0{num_qubits}b')[::-1]
    
    # Create a superposition of all possible bit orderings
    qc.h(qr)
    qc.barrier()
    
    # Check if the four numbers form a rectangle
    for i in range(num_qubits):
        qc.cx(qr[i], qr[num_qubits - 1])
        qc.ccx(qr[(i + 1) % num_qubits], qr[num_qubits - 1], qr[num_qubits - 2])
        qc.cx(qr[i], qr[num_qubits - 1])
    
    # Measure the qubits
    qc.measure(qr, cr)
    
    # Execute the circuit and get the results
    backend = Aer.get_backend('qasm_simulator')
    result = backend.run(qc).result()
    counts = result.get_counts()
    
    # Check if any rectangle was found
    for c in counts:
        if c[0] == '1':
            if int(c[1:], 2) == A + C and int(c[2:], 2) == B + D:
                return 1
            elif int(c[1:], 2) == A + D and int(c[2:], 2) == B + C:
                return 1
    
    # If no rectangle was found, return 0
    return 0

print(is_rectangle_possible(5,6,6,5))

CircuitError: 'duplicate qubit arguments'

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
import math

def is_rectangle(a, b, c, d):
    # Determine the maximum value of the four integers
    max_val = max(a, b, c, d)
    
    # Determine the number of qubits required to represent the integers
    num_qubits = math.ceil(math.log2(max_val)) + 1
    
    # Initialize the quantum and classical registers
    qr = QuantumRegister(num_qubits, 'q')
    cr = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Initialize the vector representing the four integers
    qc.initialize([1 if i == a or i == b or i == c or i == d else 0 for i in range(2 ** num_qubits)], qr)
    
    # Apply the quantum Fourier transform
    qc.h(qr)
    for i in range(num_qubits):
        for j in range(i):
            qc.cu1(math.pi / float(2 ** (i - j)), qr[j], qr[i])
        qc.h(qr[i])
    
    # Measure the Fourier coefficients
    qc.measure(qr, cr)
    
    # Simulate the circuit to obtain the measurement results
    backend = Aer.get_backend('qasm_simulator')
    result = backend.run(qc, shots=1).result()
    counts = result.get_counts()
    
    # Check if any of the Fourier coefficients are non-zero
    for key in counts:
        if counts[key] > 0 and key != '0' * num_qubits:
            return 1
    
    # If no non-zero Fourier coefficients were found, return 0
    return 0

print(is_rectangle(5,6,6,5))

QiskitError: 'Sum of amplitudes-squared does not equal one.'

In [9]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer
from qiskit.visualization import plot_histogram

def rectangle_checker(A, B, C, D):
    # Determine the maximum length of the sides
    max_side = max(A, B, C, D)
    
    # Determine the number of qubits required
    num_qubits = max_side.bit_length() + 1
    
    # Initialize the quantum circuit
    qr = QuantumRegister(num_qubits, 'q')
    cr = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Encode the lengths of the sides as binary numbers
    for i, side in enumerate([A, B, C, D]):
        for j in range(num_qubits):
            if (side >> j) & 1:
                qc.x(qr[i*num_qubits+j])
    
    # Apply the oracle to check for a rectangle
    for i in range(num_qubits):
        qc.cx(qr[i], qr[i+num_qubits])
        qc.cx(qr[i], qr[i+2*num_qubits])
        qc.cx(qr[i+num_qubits], qr[i+2*num_qubits])
    
    # Measure the output qubits
    for i in range(num_qubits):
        qc.measure(qr[i+2*num_qubits], cr[i])
    
    # Run the circuit and get the measurement counts
    backend = Aer.get_backend('qasm_simulator')
    counts = backend.run(qc).result().get_counts()
    
    # Check if a rectangle was found
    for key in counts.keys():
        if key == '1' * num_qubits:
            return 1
    
    # Return 0 if no rectangle was found
    return 0

print(rectangle_checker(5,6,6,5))

IndexError: list index out of range

In [11]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer
from qiskit.tools.monitor import job_monitor

def is_rectangle_possible_quantum(A, B, C, D):
    # Determine the maximum value of the input integers
    max_val = max(A, B, C, D)
    
    # Determine the number of qubits required to encode the input integers
    num_qubits = max_val.bit_length() + 1
    
    # Initialize the quantum circuit
    qr = QuantumRegister(num_qubits, 'q')
    cr = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Encode the input integers as binary numbers
    for i, num in enumerate([A, B, C, D]):
        for j in range(num_qubits):
            if (num >> j) & 1:
                qc.x(qr[j + i*num_qubits])
    
    # Apply a Hadamard gate to each qubit
    for i in range(num_qubits*4):
        qc.h(qr[i])
    
    # Apply the oracle to check if a rectangle is possible
    qc.cx(qr[0], qr[3*num_qubits])
    qc.cx(qr[1], qr[3*num_qubits + 1])
    qc.cx(qr[2], qr[3*num_qubits + 2])
    qc.ccx(qr[3*num_qubits], qr[3*num_qubits + 1], qr[3*num_qubits + 2])
    qc.cx(qr[0], qr[3*num_qubits])
    qc.cx(qr[1], qr[3*num_qubits + 1])
    qc.cx(qr[2], qr[3*num_qubits + 2])
    
    # Apply the inverse Hadamard gate to each qubit
    for i in range(num_qubits*4):
        qc.h(qr[i]).inverse()
    
    # Measure the qubits and extract the result
    qc.measure(qr, cr)
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(qc, shots=1)
    result = job.result().get_counts()
    return 1 if '1' in result else 0

print(is_rectangle_possible_quantum(5,6,6,5))

IndexError: list index out of range

In [12]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer
from qiskit.visualization import plot_histogram

def is_rectangle_exist(A, B, C, D):
    # Determine the maximum length of a side
    max_len = max(A, B, C, D)
    
    # Determine the number of qubits required to represent the maximum length
    num_qubits = max_len.bit_length()
    
    # Initialize the quantum circuit
    qr = QuantumRegister(num_qubits * 4, 'q')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Encode the input values as binary numbers
    for i, n in enumerate([A, B, C, D]):
        for j in range(num_qubits):
            if (n >> j) & 1:
                qc.x(qr[i*num_qubits + j])
    
    # Apply the Hadamard gate to all qubits
    qc.h(qr)
    
    # Apply the Toffoli gate to compare the first two values
    for i in range(num_qubits):
        qc.ccx(qr[i], qr[num_qubits + i], qr[2*num_qubits + i])
    
    # Apply the Toffoli gate to compare the last two values
    for i in range(num_qubits):
        qc.ccx(qr[2*num_qubits + i], qr[3*num_qubits + i], qr[4*num_qubits + i])
    
    # Measure the output qubit
    qc.measure(qr[4*num_qubits], cr)
    
    # Simulate the quantum circuit and obtain the measurement results
    backend = Aer.get_backend('qasm_simulator')
    counts = backend.run(qc).result().get_counts(qc)
    
    # Determine if a rectangle exists based on the measurement result
    if '1' in counts:
        return 1
    else:
        return 0
print(is_rectangle_exist(5,6,6,5))

IndexError: list index out of range